<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0310_HW_Ensemble%2BRandom_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **앙상블 학습 (Ensemble Learning)**

* 배깅, 부스팅 스태킹 등

## **투표 기반 분류기**

**직접 투표 (hard voting)**

* 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측하는 방식 (= 다수결 투표)
* 다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다 정확도가 높은 경우가 많음
* 각 분류기가 **약한 학습기(weak learner)**일지라도 그 종류가 많고 다양하다면 **강한 학습기(strong learner)**가 될 수 있음
    * 큰 수의 법칙 (Law of large numbers) 때문
    * 따라서 앙상블 방법은 예측기가 가능한 한 서로 독립적일 때(큰 수의 법칙의 조건) 최고의 성능을 발휘함
    * 분류기를 각기 다른 알고리즘으로 학습시킴으로써 다양한 분류기를 얻을 수 있으며, 이를 통해 앙상블 모델의 정확도를 향상시킬 수 있음


**간접 투표 (soft voting)**

* 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측하는 방식 (predict_proba() 메서드가 있는 경우)
* 확률이 높은 투표에 비중을 더 둠 >> 직접 투표 방식보다 성능이 높음
* VotingClassifier()의 voting='soft' 옵션을 통해 구현 가능

In [1]:
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드 (moons 데이터셋)
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# VotingClassifier 생성/훈련
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                             voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_pred, y_test))

LogisticRegression 0.8
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


In [3]:
# soft voting 구현
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)   # SVC : 기본값에서는 클래스 확률을 제공하지 않음 (Probability=True 옵션 설정 필요)

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                             voting='soft')
voting_clf.fit(X_train, y_train)

# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_pred, y_test))

LogisticRegression 0.8
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


## **배깅과 페이스팅 (Bagging and Pasting)**

다양한 분류기를 만드는 방법
* 각기 다른 훈련 알고리즘 사용
* 같은 알고리즘을 사용하되 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습

**배깅 (Bagging)**
* 훈련 세트에서 중복을 허용하여 샘플링하는 방식
* Bootstrap aggregating의 줄임말

**페이스팅 (Pasting)**
* 중복을 허용하지 않고 샘플링하는 방식

> 즉, 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 사용할 수 있음

![bagging&pasting](https://velog.velcdn.com/images/kyungmin1029/post/bbb0f2f0-7f96-4828-8e05-ba8f86e965ce/image.png)

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아 **새로운 샘플**에 대한 예측을 생성
* 분류의 경우, 최빈값을 채택 (= 직접투표)
* 회귀의 경우, 평균을 채택

### **사이킷런의 배깅과 페이스팅**

In [4]:
# 배깅 구현
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 500개의 앙상블 훈련, 각 분류기 중복허용 50개의 샘플로 훈련 (페이스팅 : bootstrap=False)
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=50, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

![bagging](https://velog.velcdn.com/images/kyungmin1029/post/1a691798-6072-4617-af51-d548f1335b9b/image.png)

* 앙상블 예측이 결정 트리 하나의 예측보다 일반화가 더 잘 됨
* 앙상블은 비슷한 편향에서 더 작은 분산
    * 훈련 시트의 오차 수가 거의 비슷하나 결정 경계는 덜 불규칙함

### **oob 평가**

**oob 샘플**
* out-of-bag 샘플
* 선택되지 않은 훈련 샘플을 칭함
    * 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 됨
    * 즉, 나머지 37%가 oob 샘플
* 예측기마다 남겨진 oob 샘플은 모두 다름

결국 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않음
* 따라서 별도의 검증 세트(validation set)를 사용하지 않고 oob 샘플을 사용하여 평가할 수 있음
* 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻음

In [5]:
# oob_score=True 지정 : 훈련이 끝난 후 자동으로 oob 평가 수행
# oob_score_ 변수를 통해 출력 가능
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            bootstrap=True, n_jobs=-1, oob_score=True)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.9

In [7]:
# oob 샘플에 대한 결정 함수의 값 확인
# 첫번째 훈련 세트의 경우, 음성클래스에 속할 확률을 96.87% 정도로 추정함
bag_clf.oob_decision_function_

array([[0.78461538, 0.21538462],
       [0.02051282, 0.97948718],
       [0.98863636, 0.01136364],
       [0.046875  , 0.953125  ],
       [0.0052356 , 0.9947644 ],
       [0.21649485, 0.78350515],
       [0.81656805, 0.18343195],
       [0.01104972, 0.98895028],
       [0.        , 1.        ],
       [0.08333333, 0.91666667],
       [1.        , 0.        ],
       [0.59473684, 0.40526316],
       [0.96385542, 0.03614458],
       [0.94708995, 0.05291005],
       [0.99346405, 0.00653595],
       [0.0201005 , 0.9798995 ],
       [0.92265193, 0.07734807],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.48019802, 0.51980198],
       [0.80412371, 0.19587629],
       [0.27388535, 0.72611465],
       [0.19889503, 0.80110497],
       [0.        , 1.        ],
       [0.99447514, 0.00552486],
       [1.        , 0.        ],
       [0.02659574, 0.97340426],
       [0.00995025, 0.99004975],
       [1.

## **랜덤 패치와 랜덤 서브스페이스**

이미지와 같은 고차원의 데이터셋을 다룰 때 유용

**랜덤 패치 방식**
* 훈련 특성과 샘플을 모두 샘플링하는 것

**랜덤 서브스페이스**
* 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것


```
# 랜덤 서브스페이스 구현 옵션
bootstrap = False, max_samples=1.0, bootstrap=True, max_features 1.0보다 작게 설정)
```



# **랜덤 포레스트**

배깅(또는 페이스팅)을 적용한 결정 트리의 앙상블

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입함
* 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮추어 전체적으로 더 좋은 모델을 만들어냄

In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [9]:
# BaggingClassifier로 랜덤포레스트 구현
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features='auto', max_leaf_nodes=16),
                            n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

## **엑스트라 트리**

**엑스트라 트리 (Extra-trees)**
* 익스트림 랜덤 트리 (extremely randomized trees)
* 극단적으로 무작위한 트리의 랜덤 포레스트
    * 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신(보통의 결정트리) 후보 특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택



```
ExtraTreesClassifier()
```



## **특성 중요도 (Feature Importance)**

* 사이킷런은 랜덤 포레스트에 있는 모든 트리에 걸쳐 *어떤 특성을 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지* 확인하여 특성 중요도를 측정
    * 훈련이 끝나면 자동으로 특성 중요도 점수를 계산
    * feature_importances_ 변수에 저장됨

In [11]:
# iris 데이터를 통해 확인
from sklearn.datasets import load_iris
iris = load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])

for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.10042623726159093
sepal width (cm) 0.020913805993213207
petal length (cm) 0.44255456784871616
petal width (cm) 0.43610538889647976


sepal_lenght, sepal_width (꽃잎의 길이, 너비) 피처가 중요함

*MNIST 데이터셋에 랜덤 포레스트 분류기 훈련 후 각 픽셀의 중요도 시각화*

 ![MNIST Feature Importance](https://user-images.githubusercontent.com/37871541/50535298-1066a480-0b8c-11e9-8825-0d10c12e2423.png)